[Reference](https://towardsdev.com/build-a-webhook-endpoint-with-fastapi-d14bf1b1d55d)|

In [2]:
pip install fastapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 1.7 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 11.1 MB 36.9 MB/s 
     |████████████████████████████████| 80 kB 8.6 MB/s 


In [3]:
import hashlib
import hmac
import http
import os

from fastapi import FastAPI, Header, HTTPException, Request

app = FastAPI()


def generate_hash_signature(
    secret: bytes,
    payload: bytes,
    digest_method=hashlib.sha1,
):
    return hmac.new(secret, payload, digest_method).hexdigest()


@app.post("/webhook", status_code=http.HTTPStatus.ACCEPTED)
async def webhook(request: Request, x_hub_signature: str = Header(None)):
    payload = await request.body()
    secret = os.environ.get("WEBHOOK_SECRET").encode("utf-8")
    signature = generate_hash_signature(secret, payload)
    if x_hub_signature != f"sha1={signature}":
        raise HTTPException(status_code=401, detail="Authentication error.")
    return {}